# Creating an end-to-end automated trading system

What are we going to build?
---------------------------

A day trading system for trading in National Stock Exchange(NSE).

Orders would be placed at the start of the day and closed by the end of the day or exit by a stop loss. We would be using the end of the day data as our source data. 

The trading system has four distinct parts

1. Creating a trading strategy
2. Backtesting and simulation
3. Deploying the system and order execution
4. Evaluating the performance of the system


We follow a simple principle
> **Don't waste your money**

# Primer on financial concepts

A short primer on risk and return calculations. This is more of a practical explanation; so take it with a grain of salt.
 

## Putting it simply

> ### **Risk** is the amount of money you are willing to lose

> ### **Return** is the amount of money you expect to gain

A risk to return ratio of 1:2 mean you are willing to lose 1 rupee if you expect a gain of 2 rupees.

The risk-reward ratio is often misleading since

 * there are no probabilities attached to the risk and return
 * we strictly expect either we lose 1 or gain 2 but in practice we may end up between these two cases most of the time
 
So, we would be using **payoff** to evaluate strategies




Payoff is the amount of money you would expect to receive for the amount of investment made.

It is the average returns and losses multiplied by the probability of returns and losses. Let's look at an example

Let's assume a trading strategy with a risk-reward ratio of 1:2 with the following details

 * lose 1 - 30% of the time
 * gain 2 - 20% of the time
 * makes 0 - 50% of the time
 
Then the pay off is 

`-1*0.3 + 2*0.2 + 0*0.5 = 0.1`

So, you would making 0.1 for every rupee invested.

Payoff makes decision making easier. A positive payoff is favorable while a negative payoff is unfavorable

**Warning**

We expect these probabilities in the long run based on history. So, we would very well end up in loss in the short or medium term even if our probabilities turn out to be right.

### Rule 01

> ### Risk is very very personal

### Rule 02

> ### Returns are always expectations unless realized

### Return calculations

Return calculation is similar to net profit calculation

$Profit = Sell \ Value - Buy \ Value$

$Returns = \frac{Sell Value - Buy Value}{Buy Value}$

In case of annualized returns, then

$Annualized Returns = \frac{Sell Value - Buy Value}{Buy Value} * \frac{365}{holding\ period}$

 * Holding period is the period you held the investment
 * Buy and sell value is the quantity multiplied by average price
 




Warning
=========
 > Annualized return is not the actual return (unless the holding period is exactly 365 days). We are just proportionately calculating the returns based on the holding period. So if the holding period is 2 months, we assume we would get these returns for the rest of the year, which is quite impractical. Thus we must distinguish properly between holding period returns and annualized returns.
### Rule 03

> ### Know your assumptions and always write them down

**Example**

100 units of stock A is bought on 1st JAN 2019 for 2500 and sold on 25th MAR 2019 for 2620. Calculate the returns

In [1]:
import pandas as pd

# Inputs
buy_price = 2500
buy_date = pd.to_datetime('2019-01-01')
sell_price = 2620
sell_date = pd.to_datetime('2019-03-25')
qty = 100

# Calculations
buy_value = buy_price * qty
sell_value = sell_price * qty
holding_period = (sell_date - buy_date).days # Using pandas attribute to get the days

# Outputs
profit = sell_value - buy_value
returns = (sell_value - buy_value)/buy_value
annualized_return = returns * (365/holding_period)

print('Net profit  = {}'.format(profit))
print('Returns for the period = {:.2f}%'.format(returns*100))
print('Annualized returns = {:.2f}%'.format(annualized_return*100))

Net profit  = 12000
Returns for the period = 4.80%
Annualized returns = 21.11%


In [2]:
# Exercise 01 - Calculate the returns
# 1000 units of an index bought for 11000
# and sold for 12000 after holding for 60 days


In [3]:
# Exercise 02 - Calculate the returns
# 100 units of a stock bought on 1st JAN 2018 for 3400
# and sold for 3185 on 20th JULY 2019

## Leverage

**Leverage is a multiplier on your capital**.

Leverage is borrowing money so that you can get greater gains with your investment.

**Example**

You have 10000 and you buy 100 units of stock A at 100 and sell them at 110. 

Your profit is $(110-100)*100 = 1000$

Say you have a leverage of 5. This means you now have a capital of $5*10000=50000$ to invest. Now instead of 100, you could buy 500 units of Stock A. 

Now your profit is $(110-100)*500 = 5000$ 


### How leverage works?

Leverage is akin to borrowing in regular business. You borrow 40000 and then invest your 10000 capital. After the trade, you repay the borrowing and take away all the profit. In regular business, you pay interest on this borrowing and the loans are usually provided by the banks. In stock trading, this leverage is provided by the broker, and in most of the cases you get it for free (without interest) from your broker. Your broker provides it for free since you would be increasing your turnover; so the broker benefits from brokerage. It also provides better liquidity to the market due to more turnover with lesser investment.

**Leverage is a double-edged sword.** You can go down as fast as you go up and it may not be exactly linear all the time, making it difficult to comprehend.

### Rule 04

> If you are new to markets, don't leverage

Leverage doesn't change your profits but it changes your return percentage. Your returns and annualized returns get multiplied by the leverage. In our previous example, the returns would be 

<table>
    <thead>
        <th>Leverage</th>
        <th>Own Capital</th>
        <th>Borrowed</th>
        <th>Returns</th>
        <th>Annualized returns</th>
    </thead>
    <tbody>
        <tr>
            <td>1X</td>
            <td>250000</td>
            <td>0</td>
            <td>4.80%</td>
            <td>21.11%</td>
        </tr>
          <tr>
            <td>2X</td>
            <td>125000</td>
            <td>125000</td>
            <td>9.60%</td>
            <td>42.22%</td>          
        </tr>
        <tr>
            <td>5X</td>
            <td>50000</td>
            <td>200000</td>
            <td>24.00%</td>
            <td>105.54%</td>          
        </tr>
        <tr>
            <td>10X</td>
            <td>25000</td>
            <td>225000</td>
            <td>48.00%</td>
            <td>211.10%</td>          
        </tr>
    </tbody>
    
</table>

*Unless otherwise mentioned, there would be no leverage in our system*
    

## Short selling

In stock markets, you can sell stocks you don't have. This is called **short selling** and is allowed in intraday trading. This would change the return calculation as the denominator is replaced.

$Returns = \frac{Sell Value - Buy Value}{Sell Value}$

If you buy a stock first, it is called a **LONG position.**

If you sell a stock first(short selling), its called a **SHORT position.**


## Stop Loss

Stop loss is the amount at which you exit the order if a stock moves in the opposite direction. It is a loss protection mechanism.

$Stop Loss(BUY) = price*(1-stoploss)$

$Stop Loss(SELL) = price*(1+stoploss)$

**Example**

Price of Stock A is 100. Calculate stop loss at 2% for both BUY and SELL

In [7]:
price = 100
stop_loss = 2*0.01
buy_sl = 100*(1-stop_loss)
sell_sl = 100*(1+stop_loss)

print('Stop Loss 2% if stock is bought at 100 = {}'.format(buy_sl))
print('Stop Loss 2% if stock is sold at 100 = {}'.format(sell_sl))

Stop Loss 2% if stock is bought at 100 = 98.0
Stop Loss 2% if stock is sold at 100 = 102.0


In [9]:
# Exercise 03
# The price of stock A is 3450. 
# Calculate stop loss at 3.5% for both BUY and SELL


## A few more terms

**LIMIT order**
A limit order is executed exactly at the given price.

**MARKET order**
A market order is executed at the best available price.

**Costs**
Costs are other costs related to trading such as brokerage, statutory costs etc

**Slippage**
Slippage is the difference between our expected price and the executed price

## Data Collection and storage

We would be using data from the official [NSE Website](https://nseindia.com/). Only end of the day data for capital markets would be used; this file is called bhav copy. 

You could get the historical data from this [link](https://nseindia.com/products/content/equities/equities/archieve_eq.htm). NSE provides data for all the stocks. For this tutorial, files in the data directory would be used. These files are just a selected compilation from the files downloaded from the above link.

This data is used since it is provided officially by the NSE and is available for free for personal use. Using end of data could possibly save you from [survivorship bias](https://en.wikipedia.org/wiki/Survivorship_bias).

Index data could be obtained from this [site](https://www.niftyindices.com/).

You could store this data as flat files or in a database or a HDF5 file. S